In [1]:
# importing libraries
from zipfile import ZipFile
import numpy as np
import openpyxl
import pandas as pd


In [4]:
def read_zip(year):
    # specifying the zip file directory
    file_name = f"/mnt/d/Mexico_Births/sinac_{year}.zip"
    
    # opening the zip file in READ mode
    with ZipFile(file_name, 'r') as zip:
        # listing all the contents of the zip file
        file_list = zip.filelist
        # getting a DataFrame from the file in the index 0
        data = pd.read_csv(zip.open(file_list[0].filename) )
    return data

In [48]:
# columns names for different year 
columns_20 = ["EDAD","ESTADOCONYUGAL","ESCOLARIDAD","CLAVEOCUPACIONHABITUAL","FECHANACIMIENTO",\
    "HORANACIMIENTO", "SEXO", "TALLA", "PESO", "CLUES", "ENTIDADFEDERATIVAPARTO", "MUNICIPIOPARTO",\
"LOCALIDADPARTO"]

columns_19 = ["EDADM","EDOCIVIL","NIV_ESCOL","CVEOCUPHAB","FECH_NACH",\
    "HORA_NACH", "SEXOH", "TALLAH", "PESOH", "CLUES", "ENT_NAC", "MPO_NAC",\
"LOC_NAC"]

columns_16 = ["edad_madre","estado_conyugal","escolaridad_madre","ocupacion_habitual_madre","fecha_nacimiento_nac_vivo",\
    "hora_nacimiento_nac_vivo", "sexo_nac_vivo", "talla_nac_vivo", "peso_nac_vivo", "clues", "entidad_nacimiento", "municipio_nacimiento",\
"localidad_nacimiento"]

In [44]:
# reading births_colnames file which contains tables to join on main data
file_name = f"/mnt/d/Mexico_Births/births_colnames.xlsx"
aux_data = pd.read_excel(file_name, sheet_name= None)
aux_sheets = tuple(aux_data.keys())[4:] # taking sheets to join 
print(aux_sheets) 

('estado_conyugal', 'sexo', 'escolaridad', 'ocupacion2020', 'ocupacion2019')


In [45]:
aux_data[aux_sheets[0]] # estado counyugal table

,Clave_2020,Clave_2019_2015,desc_hom_cony
0,5,11,CASADA
1,1,12,SOLTERA
2,3,13,DIVORCIADA
3,2,14,VIUDA
4,4,15,UNIÓN LIBRE
5,6,16,SEPARADA
6,0,88,NO ESPECIFICADO
7,9,99,SE IGNORA


In [51]:
aux_data[aux_sheets[1]] # genero table

,Clave_2020,Clave_2019_2015,desc_hom_sex
0,0,0,NO ESPECIFICADO
1,1,1,HOMBRE
2,2,2,MUJER
3,9,9,SE IGNORA


In [47]:
aux_data[aux_sheets[2]] # escolaridad table

,Clave_2020,Clave_2019_2015,desc_hom_esc
0,0,0,NO ESPECIFICADO
1,1,1,NINGUNA
2,31,3,PRIMARIA COMPLETA
3,32,2,PRIMARIA INCOMPLETA
4,51,5,SECUNDARIA COMPLETA
5,52,4,SECUNDARIA INCOMPLETA
6,71,7,BACHILLERATO O PREPARATORIA COMPLETA
7,72,6,BACHILLERATO O PREPARATORIA INCOMPLETA
8,81,8,LICENCIATURA O PROFESIONAL COMPLETO
9,82,11,LICENCIATURA O PROFESIONAL INCOMPLETO


In [33]:
data = read_zip(2020)

/tmp/ipykernel_11836/3653674599.py:1: DtypeWarning: Columns (49,59) have mixed types.Specify dtype option on import or set low_memory=False.
  data = read_zip(2020)


In [49]:
data = data[columns_20]

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747847 entries, 0 to 1747846
Data columns (total 13 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   EDAD                    int64 
 1   ESTADOCONYUGAL          int64 
 2   ESCOLARIDAD             int64 
 3   CLAVEOCUPACIONHABITUAL  int64 
 4   FECHANACIMIENTO         object
 5   HORANACIMIENTO          object
 6   SEXO                    int64 
 7   TALLA                   int64 
 8   PESO                    int64 
 9   CLUES                   object
 10  ENTIDADFEDERATIVAPARTO  int64 
 11  MUNICIPIOPARTO          int64 
 12  LOCALIDADPARTO          int64 
dtypes: int64(10), object(3)
memory usage: 173.4+ MB


In [52]:
data = pd.merge(data, aux_data[aux_sheets[0]], how="left", left_on="ESTADOCONYUGAL", right_on="Clave_2020")
data = pd.merge(data, aux_data[aux_sheets[1]], how="left", left_on="SEXO", right_on="Clave_2020")
data = pd.merge(data, aux_data[aux_sheets[2]], how="left", left_on="ESCOLARIDAD", right_on="Clave_2020")
data = pd.merge(data, aux_data[aux_sheets[3]], how="left", left_on="CLAVEOCUPACIONHABITUAL", right_on="Clave_2020")

/tmp/ipykernel_11836/3021495784.py:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Clave_2020_x'} in the result is deprecated and will raise a MergeError in a future version.
  data = pd.merge(data, aux_data[aux_sheets[3]], how="left", left_on="CLAVEOCUPACIONHABITUAL", right_on="Clave_2020")


In [54]:
data.columns

Index(['EDAD', 'ESTADOCONYUGAL', 'ESCOLARIDAD', 'CLAVEOCUPACIONHABITUAL',
       'FECHANACIMIENTO', 'HORANACIMIENTO', 'SEXO', 'TALLA', 'PESO', 'CLUES',
       'ENTIDADFEDERATIVAPARTO', 'MUNICIPIOPARTO', 'LOCALIDADPARTO',
       'Clave_2020_x', 'Clave_2019_2015_x', 'desc_hom_cony', 'Clave_2020_y',
       'Clave_2019_2015_y', 'desc_hom_sex', 'Clave_2020_x', 'Clave_2019_2015',
       'desc_hom_esc', 'Clave_2020_y', 'desc_hom_ocup'],
      dtype='object')

In [56]:
data.head(3)

,EDAD,ESTADOCONYUGAL,ESCOLARIDAD,CLAVEOCUPACIONHABITUAL,FECHANACIMIENTO,HORANACIMIENTO,SEXO,TALLA,PESO,CLUES,...,Clave_2019_2015_x,desc_hom_cony,Clave_2020_y,Clave_2019_2015_y,desc_hom_sex,Clave_2020_x,Clave_2019_2015,desc_hom_esc,Clave_2020_y,desc_hom_ocup
0,29,5,81,53,09/01/2020,01:38,2,50,3000,JCSMP000861,...,11.0,CASADA,2,2,MUJER,81,8,LICENCIATURA O PROFESIONAL COMPLETO,53,TRABAJADORES EN SERVICIOS DE PROTECCIÓN Y VIGI...
1,26,5,31,2,04/01/2020,05:30,1,52,9999,9998,...,11.0,CASADA,1,1,HOMBRE,31,3,PRIMARIA COMPLETA,2,"NO REMUNERADO, AMA DE CASA"
2,25,4,51,2,08/01/2020,10:51,1,51,3200,9998,...,15.0,UNIÓN LIBRE,1,1,HOMBRE,51,5,SECUNDARIA COMPLETA,2,"NO REMUNERADO, AMA DE CASA"


In [62]:
only_columns = ['EDAD', 'desc_hom_cony','desc_hom_esc','desc_hom_ocup', 'FECHANACIMIENTO', 'HORANACIMIENTO', 'desc_hom_sex', 'TALLA',\
     'PESO', 'CLUES','ENTIDADFEDERATIVAPARTO', 'MUNICIPIOPARTO', 'LOCALIDADPARTO']

In [63]:
data = data.loc[:,only_columns]

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1747847 entries, 0 to 1747846
Data columns (total 13 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   EDAD                    int64 
 1   desc_hom_cony           object
 2   desc_hom_esc            object
 3   desc_hom_ocup           object
 4   FECHANACIMIENTO         object
 5   HORANACIMIENTO          object
 6   desc_hom_sex            object
 7   TALLA                   int64 
 8   PESO                    int64 
 9   CLUES                   object
 10  ENTIDADFEDERATIVAPARTO  int64 
 11  MUNICIPIOPARTO          int64 
 12  LOCALIDADPARTO          int64 
dtypes: int64(6), object(7)
memory usage: 186.7+ MB


In [ ]:
columns_16 = ["edad_madre","estado_conyugal","escolaridad_madre","ocupacion_habitual_madre","fecha_nacimiento_nac_vivo",\
    "hora_nacimiento_nac_vivo", "sexo_nac_vivo", "talla_nac_vivo", "peso_nac_vivo", "clues", "entidad_nacimiento", "municipio_nacimiento",\
"localidad_nacimiento"]